In [ ]:
# Instalação das bibliotecas necessárias
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl pacmap datasets langchain-community ragatouille

# Carregar o dataset
import datasets
ds = datasets.load_dataset("Eitanli/goodreads", split="train")
ds = ds.select(range(500))  # Selecionar as primeiras 500 linhas
print(ds[0])  # Exibir a primeira linha para verificar


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/737 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


goodreads_data.csv:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'Unnamed: 0': 0, 'Book': 'To Kill a Mockingbird', 'Author': 'Harper Lee', 'Description': 'The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.', 'Genres': "['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young 

In [ ]:
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.notebook import tqdm

# Criar a base de conhecimento
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=str(doc["Description"]), metadata={"Author": doc["Author"], "Book": doc["Book"], "Genres": doc["Genres"]}) for doc in tqdm(ds)
]

# Processar documentos em chunks
MARKDOWN_SEPARATORS = ["\\n#{1,6} ", "```\\n", "\\n\\*\\*\\*+\\n", "\\n---+\\n", "\\n___+\\n", "\\n\\n", "\\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50, separators=MARKDOWN_SEPARATORS)
docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])
print(f"Número de documentos processados: {len(docs_processed)}")


  0%|          | 0/500 [00:00<?, ?it/s]

Número de documentos processados: 1126


In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

# Função para criar índice vetorial com modelo de embeddings especificado
def create_vector_database(docs, model_name):
    embedding_model = HuggingFaceEmbeddings(
        model_name=model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},
    )
    return FAISS.from_documents(docs, embedding_model, distance_strategy=DistanceStrategy.COSINE)

# Criar índice vetorial com modelo padrão
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
KNOWLEDGE_VECTOR_DATABASE = create_vector_database(docs_processed, EMBEDDING_MODEL_NAME)


<ipython-input-3-0016ef968c78>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install huggingface-cli

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `GABRIEL_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `GABRIEL_TOK

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

def load_llm_with_prompt(model_name):
    # Configuração do modelo em 4 bits
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    # Carregar modelo e tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Definir o formato do prompt no estilo chat
    # Instead of using apply_chat_template, manually format the prompt
    def format_prompt(context, question):
        prompt = f"""Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{context}
---
Now here is the question you need to answer.

Question: {question}"""
        return prompt

    return model, tokenizer, format_prompt # Return the formatting function





In [ ]:
import time
import tracemalloc
import json

# Função para coletar métricas (tempo e memória)
def measure_execution(func):
    def wrapper(*args, **kwargs):
        # Iniciar medição de tempo e memória
        start_time = time.time()
        tracemalloc.start()

        # Executar a função
        result = func(*args, **kwargs)

        # Parar medição
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()

        # Retornar métricas juntamente com o resultado
        metrics = {
            "execution_time": end_time - start_time,
            "memory_used": current / 10**6,
            "memory_peak": peak / 10**6,
        }
        # Exibir métricas
        print(f"Tempo de execução: {end_time - start_time:.2f} segundos")
        print(f"Memória usada: {current / 10**6:.2f} MB")
        print(f"Memória máxima: {peak / 10**6:.2f} MB")

        tracemalloc.stop()
        return result, metrics

    return wrapper

# Atualizando a função para incluir medições

@measure_execution
def answer_with_rag(
    question: str,
    llm,
    knowledge_index,
    RAG_PROMPT_TEMPLATE, # Add RAG_PROMPT_TEMPLATE as an argument
    num_retrieved_docs: int = 10,
    num_docs_final: int = 3,
):
    print("=> Recuperando documentos...")
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Manter apenas o texto

    relevant_docs = relevant_docs[:num_docs_final]

    # Construir o prompt final
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    # Call the RAG_PROMPT_TEMPLATE function to format the prompt
    prompt = RAG_PROMPT_TEMPLATE(question=question, context=context) # Call the function to get the formatted string

    print("=> Gerando resposta...")
    answer = llm(prompt)[0]["generated_text"]

    return answer, relevant_docs

# Função para fazer perguntas com base em um arquivo JSON
def questions_from_file(llm, knowledge_index, RAG_PROMPT_TEMPLATE, file_path="questions.json"): # Add RAG_PROMPT_TEMPLATE as an argument
    try:
        with open(file_path, "r") as file:
            questions = json.load(file)
    except FileNotFoundError:
        print(f"Arquivo {file_path} não encontrado. Certifique-se de que o arquivo existe.")
        return

    answers = []

    for i, question in enumerate(questions):
        print(f"\nPergunta {i+1}: {question}")
        # Pass RAG_PROMPT_TEMPLATE to answer_with_rag
        (answer, relevant_docs), metrics = answer_with_rag(question, llm, knowledge_index, RAG_PROMPT_TEMPLATE)

        # Adicionar métricas ao resultado
        answers.append({
            "question": question,
            "answer": answer,
            "docs": relevant_docs,
            "metrics": metrics
        })

        print(f"Resposta: {answer[:200]}...")
        print("Documentos relevantes:")
        for j, doc in enumerate(relevant_docs):
            print(f"Documento {j}: {doc[:200]}...")

    # Salvar respostas e métricas em um arquivo JSON
    with open("answers.json", "w") as outfile:
        json.dump(answers, outfile, indent=4)
    print("Respostas e métricas salvas em 'answers.json'.")

# Exemplo de uso
# Crie um arquivo JSON chamado 'questions.json' com perguntas
# Depois, execute:
#questions_from_file(READER_LLM, KNOWLEDGE_VECTOR_DATABASE)


In [ ]:
# Lista de modelos para testar no benchmark
model_names = ["HuggingFaceH4/zephyr-7b-beta", "meta-llama/Llama-2-13b-chat-hf","distilgpt2"]

# Executar o benchmark para cada modelo
for model_name in model_names:
    print(f"\n=== Testando o modelo: {model_name} ===")

    # Carregar o modelo atual
    model, tokenizer, RAG_PROMPT_TEMPLATE = load_llm_with_prompt(model_name)

    # Criar pipeline com o modelo carregado
    READER_LLM = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        do_sample=True,
        temperature=0.2,
        max_new_tokens=500,
    )

    # Executar as perguntas do arquivo JSON
    # Pass RAG_PROMPT_TEMPLATE to questions_from_file
    questions_from_file(READER_LLM, KNOWLEDGE_VECTOR_DATABASE, RAG_PROMPT_TEMPLATE, file_path="questions.json")

`low_cpu_mem_usage` was None, now default to True since model is quantized.



=== Testando o modelo: HuggingFaceH4/zephyr-7b-beta ===


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0



Pergunta 1: Quais livros de romance você recomenda?
=> Recuperando documentos...
=> Gerando resposta...
Tempo de execução: 44.74 segundos
Memória usada: 0.26 MB
Memória máxima: 1.93 MB
Resposta: Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the nu...
Documentos relevantes:
Documento 0: Earthy, magical, and utterly charming, this tale of family life in turn-of-the-century Mexico became a best-selling phenomenon with its winning blend of poignant romance and bittersweet wit.The number...
Documento 1: In Othello, Shakespeare creates a powerful drama of a marriage that begins with fascination (between the exotic Moor Othello and the Venetian lady Desdemona), with elopement, and with intense mutual d...
Documento 2: Simple ideas, lasting love


Falling in love is easy. Staying in love—that’s the challenge! How can you keep your relationship fre